# Mt. Everest Data Cleansing 
Silver Layer Processing

In [59]:
from pyspark.sql import functions as F, types as T, DataFrame as Fr, Column as C 

StatementMeta(, b8bbda30-6686-4024-8e10-31bba00f5e12, 61, Finished, Available, Finished)

Starting with the Expedition data, I'll take that data from the files and create a dataframe for each file.

In [33]:
df_expeditions = spark.read.format("csv").option("header","true").option("sep", ";").load("Files/expeditions.csv")
df_peaks = spark.read.format("csv").option("header","true").option("sep", ";").load("Files/peaks.csv")
df_members = spark.read.format("csv").option("header","true").option("sep", ";").load("Files/members.csv")
df_refer = spark.read.format("csv").option("header","true").option("sep", ";").load("Files/refer.csv")

StatementMeta(, b8bbda30-6686-4024-8e10-31bba00f5e12, 35, Finished, Available, Finished)

I just want to import data about Everest, so I'll start with removing all other rows from Peaks.

In [34]:
df_peaks = df_peaks.filter(df_peaks.PKNAME == 'Everest')
#display(df_peaks)

StatementMeta(, b8bbda30-6686-4024-8e10-31bba00f5e12, 36, Finished, Available, Finished)

Next I'm doing the same concerning the Expeditions, but for this I need to do three steps:
1. Rename the PEAKID column in the Expeditions dataframe
2. Join the Expeditions with the Peaks using the PEAKID column
3. Clearing from the resulting dataframe all the columns that were imported from the Peaks dataframe 

I started by renaming the column PEAKID from the Expeditions to PEAKID_EXP so that when dropping the extra columns there would not be a duplicate PEAKID column.

In [35]:
df_expeditions = df_expeditions.withColumnRenamed('PEAKID', 'PEAKID_EXP')
df_expeditions = df_expeditions.join(df_peaks, df_expeditions.PEAKID_EXP == df_peaks.PEAKID)
df_expeditions = df_expeditions.drop(*(F.col(c) for c in df_peaks.columns))
#display(df_expeditions.head(10))

StatementMeta(, b8bbda30-6686-4024-8e10-31bba00f5e12, 37, Finished, Available, Finished)

Similarly, I'm now going to do the same three steps for the Refer dataframe:
1. Rename the EXPID column in the Refer dataframe
2. Join the Refer with the Expeditions using the PEAKID column
3. Clearing from the resulting dataframe all the columns that were imported from the Expeditions dataframe 

In [36]:
df_refer = df_refer.withColumnRenamed('EXPID', 'EXPID_REF')
df_refer = df_refer.join(df_expeditions, df_refer.EXPID_REF == df_expeditions.EXPID)
df_refer = df_refer.drop(*(F.col(c) for c in df_expeditions.columns))
#display(df_refer.head(10))

StatementMeta(, b8bbda30-6686-4024-8e10-31bba00f5e12, 38, Finished, Available, Finished)

And finally I'll filter the Members dataframe as well using the Peaks dataframe. So again:
1. Rename the PEAKID column in the Members dataframe
2. Join the Members with the Peaks using the PEAKID column
3. Clearing from the resulting dataframe all the columns that were imported from the Peaks dataframe 

In [37]:
df_members = df_members.withColumnRenamed('PEAKID', 'PEAKID_MEM')
df_members = df_members.join(df_peaks, df_members.PEAKID_MEM == df_peaks.PEAKID)
df_members = df_members.drop(*(F.col(c) for c in df_peaks.columns))
#display(df_members.head(10))

StatementMeta(, b8bbda30-6686-4024-8e10-31bba00f5e12, 39, Finished, Available, Finished)

Next up I'm going to treat the weather data, starting with the current weather data.

The values here are stored in a "key: value" format, so I need to extract each value in order to create a table with the columns I'm interested in.

It's important to note that the Time column on this dataframe is considering the timezone in Mount Everest, which is GMT + 8h.

In [38]:
df = spark.read.option("multiline", "true").json("Files/mteverest_weather_data.json")
df_current_weather = (df.select(
    F.col('current.time').alias('Datetime'),
    F.col('daily.sunrise').getItem(0).alias('Sunrise'),
    F.col('daily.sunset').getItem(0).alias('Sunset'),
    F.col('latitude').alias('Latitude'),
    F.col('longitude').alias('Longitude'),
    F.col('elevation').alias('Elevation'),
    F.col('timezone').alias('Location'),
    F.col('timezone_abbreviation').alias('Timezone'),
    F.col('current.weather_code').alias('Weather_code'),
    F.col('current.precipitation').alias('Precipitation'),
    F.col('current_units.precipitation').alias('Precipitation_unit'),  
    F.col('current.temperature_2m').alias('Temperature'),
    F.col('current_units.temperature_2m').alias('Temperature_unit'),
    F.col('current.wind_speed_10m').alias('Wind_speed'),
    F.col('current_units.wind_speed_10m').alias('Wind_speed_unit'),
    F.col('current.wind_gusts_10m').alias('Wind_gusts'),
    F.col('current_units.wind_gusts_10m').alias('Wind_gusts_unit'),
    F.col('current.relative_humidity_2m').alias('Relative_humidity'),
    F.col('current_units.relative_humidity_2m').alias('Relative_humidity_unit'),
    F.col('current.snowfall').alias('Snowfall'),
    F.col('current_units.snowfall').alias('Snowfall_unit')
))
#display(df_current_weather)

StatementMeta(, b8bbda30-6686-4024-8e10-31bba00f5e12, 40, Finished, Available, Finished)

And finally I'm organizing the information for historical weather data. I need to expand the values in different columns because each column had the results stored for all dates and I need to convert that to multiple rows.

In [60]:
df = spark.read.option("multiline", "true").json("Files/mteverest_hist_weather_data.json")

StatementMeta(, b8bbda30-6686-4024-8e10-31bba00f5e12, 62, Finished, Available, Finished)

In [61]:
df = df.select(
    F.col('hourly.time').alias('Datetime'),
    F.col('daily.sunrise').alias('Sunrise'),
    F.col('daily.sunset').alias('Sunset'),
    F.col('latitude').alias('Latitude'),
    F.col('longitude').alias('Longitude'),
    F.col('elevation').alias('Elevation'),
    F.col('timezone').alias('Location'),
    F.col('timezone_abbreviation').alias('Timezone'),
    F.col('hourly.weather_code').alias('Weather_code'),
    F.col('hourly.precipitation').alias('Precipitation'),
    F.col('hourly_units.precipitation').alias('Precipitation_unit'),  
    F.col('hourly.temperature_2m').alias('Temperature'),
    F.col('hourly_units.temperature_2m').alias('Temperature_unit'),
    F.col('hourly.wind_speed_10m').alias('Wind_speed'),
    F.col('hourly_units.wind_speed_10m').alias('Wind_speed_unit'),
    F.col('hourly.wind_gusts_10m').alias('Wind_gusts'),
    F.col('hourly_units.wind_gusts_10m').alias('Wind_gusts_unit'),
    F.col('hourly.relative_humidity_2m').alias('Relative_humidity'),
    F.col('hourly_units.relative_humidity_2m').alias('Relative_humidity_unit'),
    F.col('hourly.snowfall').alias('Snowfall'),
    F.col('hourly_units.snowfall').alias('Snowfall_unit')
)

StatementMeta(, b8bbda30-6686-4024-8e10-31bba00f5e12, 63, Finished, Available, Finished)

In [62]:
display(df)

StatementMeta(, b8bbda30-6686-4024-8e10-31bba00f5e12, 64, Finished, Available, Finished)

SynapseWidget(Synapse.DataFrame, dfc3a192-8590-493d-a7b2-8c02d60c226a)

In [65]:
df_hist_weather = (
    df
    .withColumn(
        "zipped",
        F.arrays_zip(
            F.col("Datetime"),
            F.col('Weather_code'),
            F.col("Precipitation"),
            F.col("Temperature"),
            F.col('Wind_speed'),
            F.col('Wind_gusts'),
            F.col('Relative_humidity'),
            F.col('Snowfall')
        )
    )
    .withColumn("zipped", F.explode(F.col("zipped")))
    .select(
        F.col("zipped.Datetime").alias("Datetime"),
        F.col("Latitude"),
        F.col("Longitude"),
        F.col("Elevation"),
        F.col("Location"),
        F.col("Timezone"),
        F.col('zipped.Weather_code'),
        F.col("zipped.Precipitation").alias("Precipitation"),
        F.col("Precipitation_unit"),
        F.col("zipped.Temperature").alias("Temperature"),
        F.col("Temperature_unit"),
        F.col("zipped.Wind_speed").alias("Wind_speed"),
        F.col("Wind_speed_unit"),
        F.col("zipped.Wind_gusts").alias("Wind_gusts"),
        F.col("Wind_gusts_unit"),
        F.col("zipped.Relative_humidity").alias("Relative_humidity"),
        F.col("Relative_humidity_unit"),
        F.col("zipped.Snowfall").alias("Snowfall"),
        F.col("Snowfall_unit")
    )
)
#display(df_hist_weather.head(10))

StatementMeta(, b8bbda30-6686-4024-8e10-31bba00f5e12, 67, Finished, Available, Finished)

In [66]:
display(df_hist_weather)

StatementMeta(, b8bbda30-6686-4024-8e10-31bba00f5e12, 68, Finished, Available, Finished)

SynapseWidget(Synapse.DataFrame, c8197455-8d87-44ea-bd5f-6e05575c1752)

I noticed that there were some NULL values for Date, Sunrise and Sunset columns.

In [1]:
df_hist_weather.filter(F.col("Date").isNull()).limit(1).count() > 0

StatementMeta(, , -1, Cancelled, , Cancelled)

So I need to remove those rows before loading the data into the gold layer.

In [ ]:
df_hist_weather = df_hist_weather.filter(df_hist_weather.Date.isNotNull())

Running the check again for any of those columns returns a 'False' result, so we can move forward.

Finally, I'm saving this information as a table for the gold layer.

In [ ]:
df_expeditions.write.mode('overwrite').saveAsTable('expeditions_silver')
df_peaks.write.mode('overwrite').saveAsTable('peaks_silver')
df_refer.write.mode('overwrite').saveAsTable('references_silver')
df_members.write.mode('overwrite').saveAsTable('members_silver')
df_current_weather.write.mode('overwrite').saveAsTable('weather_current_silver')
df_hist_weather.write.mode('overwrite').saveAsTable('weather_hist_silver')